<a href="https://colab.research.google.com/github/Jarmos-san/PumpItUp/blob/master/PumpItUp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [DrivenData Competition | Pump it Up: Data Mining the Water Table](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/)

Goal - ***To predict the operating condition of a waterpoint for each record in the dataset.***

[Discussion Forums](https://community.drivendata.org/c/pump-it-up-data-mining-the-water-table)

---

Problem Description has 3 sections:

- Features
- Labels
- Submission Format

## Features:

- `amount_tsh` - **Total static head (amount water available to waterpoint)**
- `date_recorded` - **The date the row was entered**
- `funder` - **Who funded the well**
- `gps_height` - **Altitude of the well**
- `installer` - **Organization that installed the well**
- `longitude` - **GPS coordinate**
- `latitude` - **GPS coordinate**
- `wpt_name` - **Name of the waterpoint if there is one**
- `num_private` -
- `basin` - **Geographic water basin**
- `subvillage` - **Geographic location**
- `region` - **Geographic location**
- `region_code` - **Geographic location (coded)**
- `district_code` - **Geographic location (coded)**
- `lga` - **Geographic location**
- `ward` - **Geographic location**
- `population` - **Population around the well**
- `public_meeting` - **True/False**
- `recorded_by` - **Group entering this row of data**
- `scheme_management` - **Who operates the waterpoint**
- `scheme_name` - **Who operates the waterpoint**
- `permit` - **If the waterpoint is permitted**
- `construction_year` - **Year the waterpoint was constructed**
- `extraction_type` - **The kind of extraction the waterpoint uses**
- `extraction_type_group` - **The kind of extraction the waterpoint uses**
- `extraction_type_class` - **The kind of extraction the waterpoint uses**
- `management` - **How the waterpoint is managed**
- `management_group` - **How the waterpoint is managed**
- `payment` - **What the water costs**
- `payment_type` - **What the water costs**
- `water_quality` - **The quality of the water**
- `quality_group` - **The quality of the water**
- `quantity` - **The quantity of water**
- `quantity_group` - **The quantity of water**
- `source` - **The source of the water**
- `source_type` - **The source of the water**
- `source_class` - **The source of the water**
- `waterpoint_type` - **The kind of waterpoint**
- `waterpoint_type_group` - **The kind of waterpoint**

## Labels:

- `functional` - **the waterpoint is operational and there are no repairs needed**
- `functional needs repair` - **the waterpoint is operational, but needs repairs**
- `non functional` - **the waterpoint is not operational**

## Submission Format:

id 	  | status_group |
------|--------------|
50785 |	functional   |
51630 |	functional   |
17168 |	functional   |
45559 |	functional   |
49871 |	functional   |

## Data Download:

File 	                                                                                                            | Description                                       |
------------------------------------------------------------------------------------------------------------------|---------------------------------------------------|
[Submission format](https://s3.amazonaws.com/drivendata/data/7/public/SubmissionFormat.csv)                       |	The format for submitting your predictions        |
[Test Set Values](https://s3.amazonaws.com/drivendata/data/7/public/702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv)     |	The independent variables that need predictions   |
[Training Set Labels](https://s3.amazonaws.com/drivendata/data/7/public/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv) |	The dependent variable (status_group) for each of the rows in Training set values   |
[Training Set Values](https://s3.amazonaws.com/drivendata/data/7/public/4910797b-ee55-40a7-8668-10efd5c1b960.csv) | The independent variables for the training set    |

In [6]:
# Loading Pandas 0.25.3 into the system along with the required files
import pandas as pd

# Checking the version for easier debugging at some point of time.
pd.__version__

'0.25.3'

In [0]:
# Load .csv file directly from the provided links
SubmissionFormat = pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/SubmissionFormat.csv')
TestSet = pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv')
TrainLabels = pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv')
TrainSet = pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/4910797b-ee55-40a7-8668-10efd5c1b960.csv')

In [14]:
print(f'Size of Labels set: {TrainLabels.shape}')
print(f'Size of Labels set: {TrainSet.shape}')

Size of Labels set: (59400, 2)
Size of Labels set: (59400, 40)


In [0]:
TrainSet

# Rough Overview

- Trainning Set contains ~60k variables.
- The `status_group` variable in the `labels.csv` needs to be formatted properly for more readability.
- The rest of the other independent variables in the Training Set require reformatting as well.
- `labels.csv` needs to be appended to the Training Set in context to the proper ID numbers.

- Read more on dataframe manipulation and i

In [0]:
# Testing